In [ ]:
# Step 1: Zensvi will create 2 type of segmented images, we only need the colored ones. 
# So this process will remove the blend images and move the colored segmented images to a new folder called "segment".
import os
import shutil
from pathlib import Path
from tqdm import tqdm

base_dir = Path("GoogleStreetViews/YOUR_OWN_CITY_NAME_year/Seg")	#replace with your city name and year, like SF_2023/Seg
segment_dir = base_dir / "segment"
segment_dir.mkdir(exist_ok=True)

for img_path in tqdm(base_dir.glob("*.png")):
    filename = img_path.name
    if filename.endswith("_blend.png"):
        img_path.unlink()  
    elif filename.endswith("_colored_segmented.png"):
        new_name = filename.replace("_colored_segmented.png", ".png")
        new_path = segment_dir / new_name
        shutil.move(str(img_path), str(new_path))


35778it [00:03, 9119.60it/s]


In [ ]:
# Step 2, run the following 3 blocks
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm
import random
from matplotlib import pyplot as plt
from concurrent.futures import ThreadPoolExecutor, as_completed

#Please check the following folder, IMG_FOLDER should target to your segmented images
IMG_FOLDER = Path("GoogleStreetViews/YOUR_OWN_CITY_NAME_year/Seg/segment")  # replace with your city name and year, like SF_2023/Seg/segment
OUT_FOLDER = Path("GoogleStreetViews/YOUR_OWN_CITY_NAME_year/YOUR_OWN_CITY_NAME_year_fisheye_512")    #replace with your city name and year, like SF_2023/SF_2023_fisheye512
OUT_FOLDER.mkdir(parents=True, exist_ok=True)


In [ ]:
def map_projection_upper(r_norm, projection="equisolid"):
    r_norm = np.clip(r_norm, 0, 1)
    if projection == "equidistant":                 # r = f θ
        theta = r_norm * (np.pi / 2)
    elif projection == "orthographic":              # r = f sin θ
        theta = np.arcsin(r_norm)
    elif projection == "equisolid":                 # r = 2f sin(θ/2)
        theta = 2 * np.arcsin(r_norm / np.sqrt(2))
    elif projection == "stereographic":             # r = 2f tan(θ/2)
        theta = 2 * np.arctan(r_norm)
    else:
        raise ValueError(f"Unsupported projection: {projection}")
    return theta   # 0–π/2


def cyl2fis_upper(img: np.ndarray, out_d: int = 512, projection: str = "equisolid") -> np.ndarray:
    
    Hc, Wc = img.shape[:2]          
    r0_src = Wc / (2 * np.pi)       
    Wf0 = Hf0 = int(np.round(Wc / np.pi))  
    scale = out_d / Wf0             
    r0_dst = r0_src * scale

    xv, yv = np.meshgrid(np.arange(out_d), np.arange(out_d))
    Cx = Cy = r0_dst
    dx, dy = xv - Cx, yv - Cy
    r_dst = np.sqrt(dx**2 + dy**2)

    mask = r_dst <= r0_dst        
    r_dst_clipped = np.where(mask, r_dst, r0_dst)   

    
    r_src = r_dst_clipped / scale
    r_norm = r_src / r0_src
    phi = (np.arctan2(dy, dx) + 2 * np.pi) % (2 * np.pi)  # 0–2π
    theta = map_projection_upper(r_norm, projection)      # 0–π/2

    # Cylindrical 
    xc = (phi / (2 * np.pi)) * Wc
    yc = (theta / (np.pi / 2)) * (Hc / 2)                 # Upper half only

    # OpenCV remap
    map_x = xc.astype(np.float32)
    map_y = yc.astype(np.float32)
    fisheye = cv2.remap(
        img, map_x, map_y,
        interpolation=cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_CONSTANT
    )

    fisheye[~mask] = 0   # fill non-fisheye pixels with black
    return fisheye


In [ ]:
def process_one_image(pano_path):
    try:
        pano = cv2.imread(str(pano_path))  # BGR
        if pano is None:
            return f"Cannot read {pano_path.name}"
        fisheye = cyl2fis_upper(pano[..., ::-1], out_d=512, projection="equisolid")
        out_path = OUT_FOLDER / pano_path.name
        cv2.imwrite(str(out_path), fisheye[..., ::-1])
    except Exception as e:
        return f"{pano_path.name} failed: {e}"
    
images = sorted([p for p in IMG_FOLDER.glob("*.jpg")] + [p for p in IMG_FOLDER.glob("*.png")])
print(f"Total panoramas detected: {len(images)}")

with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    futures = [executor.submit(process_one_image, path) for path in images]
    for f in tqdm(as_completed(futures), total=len(futures)):
        result = f.result()
        if result:
            print(result)


Total panoramas detected: 17889


100%|██████████| 17889/17889 [01:48<00:00, 164.91it/s]


In [ ]:
# Step 3: Calculate the ratio of each class in the segmented images
import os 
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed


base_dir = Path("GoogleStreetViews/YOUR_OWN_CITY_NAME_year/YOUR_OWN_CITY_NAME_year_fisheye_512")	# replace with your city name and year, like SF_2023/SF_2023_fisheye512
input_folder = base_dir  
original_csv = Path("YOUR_OWN_CITY_NAME_panorama_YEAR.csv")	# replace with your original panorama CSV file, like SF_panorama_2023.csv
merged_output_csv = base_dir / f"fisheye_result_YEAR.csv" # replace with your desired output CSV file, like fisheye_result_2023.csv

# palette for class mapping
palette = np.array([
    [128, 64,128],  # road
    [244, 35,232],  # sidewalk
    [ 70, 70, 70],  # building
    [102,102,156],  # wall
    [190,153,153],  # fence
    [153,153,153],  # pole
    [250,170, 30],  # traffic light
    [220,220,  0],  # traffic sign
    [107,142, 35],  # vegetation (tree)
    [152,251,152],  # terrain
    [ 70,130,180],  # sky
    [220, 20, 60],  # person
    [255,  0,  0],  # rider
    [  0,  0,142],  # car
    [  0,  0, 70],  # truck
    [  0, 60,100],  # bus
    [  0, 80,100],  # train
    [  0,  0,230],  # motorcycle
    [119, 11, 32],  # bicycle
], dtype=np.uint8)

CLASS_NAME = {
    'building': 2,
    'tree': 8,
    'sky': 10
}

def mask_to_class(mask):
    flat_mask = mask.reshape(-1, 3)
    class_mask = np.zeros(flat_mask.shape[0], dtype=np.uint8)
    for class_index, color in enumerate(palette):
        matches = np.all(flat_mask == color, axis=1)
        class_mask[matches] = class_index
    return class_mask.reshape(mask.shape[:2])

def calculate_ratios(class_mask):
    total_pixels = class_mask.size
    ratios = {}
    for name, index in CLASS_NAME.items():
        count = np.sum(class_mask == index)
        ratios[name] = count / total_pixels if total_pixels > 0 else 0
    return ratios

def process_image(filename):
    image_path = input_folder / filename
    try:
        image = cv2.imread(str(image_path))
        if image is None:
            raise ValueError(f"Cannot read image: {image_path}")
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        class_mask = mask_to_class(image_rgb)
        ratios = calculate_ratios(class_mask)

        pid = os.path.splitext(filename)[0]
        if pid.endswith('_colored_segmented'):
            pid = pid.rsplit('_colored_segmented', 1)[0]

        return {
            'pid': pid,
            **ratios
        }
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

def process_all_images():
    results = []
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        futures = [executor.submit(process_image, filename) for filename in files]

        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing images"):
            result = future.result()
            if result is not None:
                results.append(result)
    return results

if __name__ == '__main__':
    results = process_all_images()

    results_df = pd.DataFrame(results)
    original_df = pd.read_csv(original_csv)
    merged_df = original_df.merge(results_df, on='pid', how='left')
    merged_df = merged_df.dropna()
    
    merged_df.to_csv(merged_output_csv, index=False)
    print(f'CSV 合并完成，结果已保存到 {merged_output_csv}')


Processing images: 100%|██████████| 17889/17889 [03:05<00:00, 96.25it/s] 


CSV 合并完成，结果已保存到 /root/autodl-tmp/Zensvi/SF_seg_2013/SF_512/fisheye_result_2013.csv
